In [1]:
# Instalación de librerías requeridas
%pip install sqlalchemy
%pip install psycopg2
%pip install psycopg2-binary
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


In [3]:
with open('../../configBD/config.yml', 'r') as f:
    cfg      = yaml.safe_load(f)
    cfg_etl  = cfg['bodega']
    cfg_bd   = cfg['mensajeria']
cfg_etl        # verificación rápida


{'driver': 'postgresql+psycopg2',
 'host': 'proyectobodega.postgres.database.azure.com',
 'port': 5432,
 'user': 'adminbodega',
 'password': 'Goddess9039#',
 'db': 'proyectobodega'}

In [4]:
url_bd  = f"{cfg_bd['driver']}://{cfg_bd['user']}:{cfg_bd['password']}@{cfg_bd['host']}:{cfg_bd['port']}/{cfg_bd['db']}"
url_etl = f"{cfg_etl['driver']}://{cfg_etl['user']}:{cfg_etl['password']}@{cfg_etl['host']}:{cfg_etl['port']}/{cfg_etl['db']}"


In [5]:
cliente_bd  = create_engine(url_bd)
cliente_etl = create_engine(url_etl)


In [6]:
fact_serv = pd.read_sql_table('fact_servicios', url_etl)
dim_tiempo = pd.read_sql_table('dim_tiempo', url_etl) \
               .rename(columns={'Año':'year','Mes':'month','Dia':'day'})
dim_tiempo['date'] = pd.to_datetime(dim_tiempo[['year','month','day']]).dt.date


In [10]:
fact_serv = pd.read_sql_table('fact_servicios', url_etl)


In [11]:
# Duraciones (en minutos); usa .dt.total_seconds()/60
fact_serv['Duracion_Total'] = (
    pd.to_datetime(fact_serv['Tiempo_Cerrado'])  -
    pd.to_datetime(fact_serv['Tiempo_Inicio'])
).dt.total_seconds().div(60)

fact_serv['Duracion_Origen→Destino'] = (
    pd.to_datetime(fact_serv['Tiempo_Entregado_Destino']) -
    pd.to_datetime(fact_serv['Tiempo_Recogido_Origen'])
).dt.total_seconds().div(60)

# Convierte booleanos a int
fact_serv['EsFinal'] = fact_serv['EsFinal'].astype(int)

# Selecciona variables numéricas y categóricas
num_cols  = ['Duracion_Total','Duracion_Origen→Destino']
cat_cols  = ['TipoServicio','EstadoServicio']

# Preprocesador: escalado para numéricas + one-hot para categóricas
pre = ColumnTransformer([
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
     ])


In [19]:
# sección 9 – ingeniería de variables y preprocesamiento

# 1) selecciona sólo las columnas que usará el transformer
feature_cols = num_cols + cat_cols
df_X = fact_serv[feature_cols].copy()

# 2) imputa los NaN en las numéricas con 0 (o la estrategia que prefieras)
df_X[num_cols] = df_X[num_cols].fillna(0)

# 3) garantiza que todos los nombres de columna sean str
df_X.columns = df_X.columns.map(str)

# 4) aplica el transformer sobre el DataFrame limpio
X = pre.fit_transform(df_X)


In [20]:
# sección 10 – K-means
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
fact_serv['cluster'] = kmeans.fit_predict(X)

cluster_names = {
    0:'express',
    1:'largo_con_retraso',
    2:'fallido_cancelado',
    3:'standard',
    4:'otros'
}
fact_serv['categoria'] = fact_serv['cluster'].map(cluster_names)


e:\Univalle\13. Séptimo Semestre (Repetición)\Introd Ciencias de los datos\Clase 8\ETL\venv\Lib\site-packages\sklearn\base.py:1363: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [21]:
# 1) fecha del servicio
fact_serv['fecha_servicio'] = pd.to_datetime(fact_serv['Tiempo_Inicio']).dt.date

# 2) conteo por fecha y categoría
servicios_agr = (
    fact_serv.groupby(['fecha_servicio','categoria'])
             .size()
             .unstack(fill_value=0)
             .reset_index()
)

# 3) añadir TiempoKey
servicios_agr = servicios_agr.merge(
    dim_tiempo[['tiempo_key','date']],
    left_on='fecha_servicio', right_on='date', how='left'
).rename(columns={'tiempo_key':'TiempoKey'}).drop(columns=['date','fecha_servicio'])

# 4) guardar en DW
servicios_agr.to_sql('fact_servicios_clustering',
                     cliente_etl,
                     if_exists='replace',
                     index=False)

# 5) opcional: exportar CSV
# servicios_agr.to_csv('servicios_agrupados_clustering.csv', index=False)

servicios_agr.head()


,express,largo_con_retraso,TiempoKey
0,864,136,7152
1,864,136,7153
2,864,136,7154
3,864,136,7155
4,864,136,7156
